## Подключение зависимостей

In [1]:
pip install requests beautifulsoup4 markdown2 pandas gradio

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# Устанавливаем пакеты, необходимые для обращения к YandexGPT
%pip uninstall jwt
%pip install PyJWT -U

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import requests, re, base64, json, os, time, jwt, markdown2, streamlit

from bs4 import BeautifulSoup

# Получение ссылок, парсинг XML-ответа YandexSearch API

In [4]:
def get_urls_from_yandex(query, folder_id, api_key):
    # Формирование URL запроса к API Яндекса
    url = f"https://yandex.ru/search/xml?folderid={folder_id}&apikey={api_key}&query={query}"

    response = requests.get(url)

    if response.status_code != 200:
        raise Exception(f"ошибка при выполнении запроса: {response.status_code}")

    # Сохранение ответа в формате XML
    xml_data = response.text
    
    # Парсинг XML
    soup = BeautifulSoup(xml_data, 'xml')

    # Извлечение первых 10 URL-адресов
    urls = [url_tag.text for url_tag in soup.find_all('url')[:10]]

    return urls

folder_id = 'b1g60k1kva2jrejf255o'
api_key = os.environ['YandexSearch-API-Key'] # secret YandexSearch-API-Key from environment variables
model = 'MODEL'
query = f'технические характеристики {model}'
#OFF urls = get_urls_from_yandex(query, folder_id, api_key)
#print(urls)


## Парсинг сайтов в текстовый вид

In [5]:
# Функция парсинга html-страниц в текст
def parse_pages(urls):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    texts = []
    for url in urls:

        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            text = soup.get_text(separator=' ', strip=True)
            texts.append(text)
        else:
            texts.append("не удалось загрузить страницу)))")
    return texts
#OFF texts = parse_pages(urls)

#Вывод текста со всех страниц:

# for i, text in enumerate(texts):
#      print(f"\nТекст страницы {i+1}:\n", text)


## Словарь с характеристиками ноутбуков (tests)

In [6]:
specs =  { "ExpressCard": None, "PartNumber/Артикул Производителя": None, "Touch bar": None, "Акустическая система": None, "Антибактериальное покрытие": None, "Базовая модель": None, "Батарея": None, "Бренд": None, "Веб-камера": None, "Версия Bluetooth": None, "Версия операционной системы": None, "Вес товара": None, "Вес упаковки (ед)": None, "Видеопамять дискретного контроллера": None, "Видеопамять дискретного контроллера ГБ": None, "Влагозащитный корпус": None, "Внешняя поверхность": None, "Возможность апгрейда": None, "Возможность апгрейда HDD/SSD": None, "Возможность апгрейда оперативной памяти": None, "Возможность зарядки от Type-C (USB Power Delivery)": None, "Возможность передачи видео через USB Type-C": None, "Встроенная солнечная батарея": None, "Встроенный TV-тюнер": None, "Встроенный кард-ридер": None, "Встроенный микрофон": None, "Встроенный сабвуфер": None, "Входное напряжение (макс)": None, "Входное напряжение (мин)": None, "Высота (максимальная)": None, "Высота (минимальная)": None, "Высота упаковки (ед)": None, "Габариты упаковки (ед) ДхШхВ": None, "Глубина товара": None, "Год релиза": None, "Диагональ экрана в дюймах": None, "Длина упаковки (ед)": None, "Для геймеров": None, "Для дизайна": None, "Для работы": None, "Для учебы": None, "Док-станция в комплекте": None, "Жесткие диски (подбор)": None, "Клавиатура с русскими буквами": None, "Клавиатура только с латинскими буквами": None, "Код гарантии": None, "Количество HDD": None, "Количество SSD": None, "Количество высокопроизводительных ядер": None, "Количество графических контроллеров": None, "Количество динамиков": None, "Количество логистических мест": None, "Количество микрофонов": None, "Количество разъемов жесткого диска": None, "Количество энергоэффективных ядер": None, "Количество ядер процессора": None, "Мак. скорость передачи данных (Type-A)": None, "Мак. скорость передачи данных (Type-C)": None, "Максимальное время работы от батареи": None, "Максимальное разрешение внешнего монитора": None, "Максимальное число потоков": None, "Материал для фильтра Ситилинк": None, "Материал корпуса": None, "Модель": None, "Модель граф. контроллера": None, "Модем": None, "Мультитач": None, "Мышь в комплекте": None, "Настройка подсветки клавиатуры": None, "Общее количество портов USB Type-A": None, "Общее количество портов USB Type-С": None, "Объем HDD": None, "Объем SSD": None, "Объем кэша L2": None, "Объем кэша L3": None, "Объем оперативной памяти": None, "Объем первого SSD": None, "Объем упаковки (ед)": None, "Оперативная память": None, "Операционная система": None, "Операционная система ноутбука": None, "Особенности": None, "Особенности корпуса": None, "Параллельный импорт": None, "Плотность пикселей": None, "Поверхность экрана": None, "Поворотный экран (Tablet PC)": None, "Поддержка HDR": None, "Поддержка технологии 3G": None, "Поддержка технологии Bluetooth": None, "Поддержка технологии WiDi": None, "Поддержка технологии Wi-Fi": None, "Поддержка технологии WiMAX": None, "Поддержка языков": None, "Подсветка клавиш клавиатуры": None, "Порты e-SATA/USB": None, "Порты USB 2.0 (Type-A)": None, "Порты USB 3.0 (Type-A)": None, "Последовательный порт (RS-232)": None, "Производитель товара": None, "Процессор": None, "Процессор (для автозаполнения)": None, "Процессор, модель": None, "Процессор, производитель": None, "Процессор, серия": None, "Процессор, частота": None, "Разблокировка по лицу": None, "Размер (ноутбуки)": None, "Размеры": None, "Размеры корпуса": None, "Разрешение": None, "Разрешение (для фильтра Ситилинк)": None, "Разрешение веб-камеры": None, "Разрешение матрицы экрана": None, "Разр-ние экрана, верт-ное": None, "Разр-ние экрана, гор-ное": None, "Разрядность ОС": None, "Разъем DisplayPort": None, "Разъем D-Sub": None, "Разъем DVI": None, "Разъем e-SATA": None, "Разъем HDMI": None, "Разъем IEEE 1394 (Firewire)": None, "Разъем micro HDMI": None, "Разъем mini DisplayPort": None, "Разъем mini DVI": None, "Разъем mini HDMI": None, "Разъем для микрофона": None, "Разъем для наушников": None, "Разъем наушники/микрофон": None, "Разъем порт-репликатора": None, "Разъемов Thunderbolt": None, "Разъемов Thunderbolt 2": None, "Разъемов Thunderbolt 3": None, "Распаянная оперативная память": None, "Светодиодная подсветка экрана": None, "Семейство": None, "Сенсорный экран": None, "Серия (для фильтра Ситилинка)": None, "Серия граф. контроллера": None, "Сканер отпечатка пальца": None, "Слот замка безопасности": None, "Соотношение сторон": None, "Срок службы товара": None, "Ссылка на сайт поставщика/вендора": None, "Стандарт Wi-Fi": None, "Стилус в комплекте": None, "Сумка в комплекте": None, "Техпроцесс / литография": None, "Тип": None, "Тип ODD": None, "Тип батареи": None, "Тип видеопамяти": None, "Тип вилки": None, "Тип графического контроллера": None, "Тип кабельной сети (разъем RJ-45)": None, "Тип матрицы": None, "Тип матрицы (как указывает производитель)": None, "Тип оперативной памяти": None, "Трансформер": None, "Угол обзора": None, "Ударопрочный корпус": None, "Устройства позиционирования": None, "Функция Мультиэкран": None, "Функция быстрой зарядки": None, "Цвет (маркетинговый)": None, "Цвет для фильтра": None, "Цвет клавиатуры ноутбука": None, "Цвет подсветки клавиатуры": None, "Цветовое решение": None, "Цифровой блок клавиатуры": None, "Частота обновления": None, "Частота оперативной памяти": None, "Число ядер GPU": None, "Ширина товара": None, "Ширина упаковки (ед)": None, "Широкоформатный экран": None, "Экран с поддержкой 3D": None, "Энергоемкость батареи (Wh)": None, "Язык клавиатуры": None, "Яркость экрана": None, "Видеокарта": None, "Внутренняя поверхность": None, "Второй экран": None, "Выходная мощность адаптера питания": None, "Динамический экран": None, "Дополнительно в комплекте": None, "Количество слотов под модули памяти": None, "Поддержка NFC": None, "Пропускная способность памяти": None, "Процессор, частота (в режиме turbo)": None, "Разъемов Thunderbolt 4": None, "Режим защиты зрения": None, "Среднее время работы": None, "Технология HDR": None, "Комплектация": None, "Дополнительные технологии": None, "Контраст к 1": None, "Поддержка карт памяти": None, "Ядро процессора": None, "Батарея (cell)": None, "Версия HDMI": None, "Ёмкость батареи (mAh)": None, "Интерфейс первого разъема установленного SSD": None, "Количество свободных разъемов жесткого диска": None, "Мощность одного динамика": None, "Напряжение батареи (V)": None, "Первый разъем возможного апгрейда": None, "Порты USB 3.0 (Type-C)": None, "Порты USB 3.1 (Type-A)": None, "Порты USB 3.1 (Type-C)": None, "Разъем SSD": None, "Тип веб-камеры": None, "Форм-фактор первого разъема возможного апгрейда": None, "Форм-фактора первого разъема установленного SSD": None, "Цветовой охват sRGB": None, "Интерфейс первого разъема возможного апгрейда": None, "Относительная площадь экрана": None, "Охлаждение": None, "Особенности жестких дисков": None, "Произведено в РФ": None, "Цветовой охват NTSC": None, "Порты USB 2.0 (Type-C)": None, "Количество занятых слотов оперативной памяти": None, "Количество свободных слотов оперативной памяти": None, "Макс. объем оперативной памяти": None, "Описание возможностей апгрейда": None, "Поддержка технологии 4G": None, "Порты USB 3.2 (Type-A)": None, "Порты USB 3.2 (Type-C)": None, }

In [7]:
specs.keys()

dict_keys(['ExpressCard', 'PartNumber/Артикул Производителя', 'Touch bar', 'Акустическая система', 'Антибактериальное покрытие', 'Базовая модель', 'Батарея', 'Бренд', 'Веб-камера', 'Версия Bluetooth', 'Версия операционной системы', 'Вес товара', 'Вес упаковки (ед)', 'Видеопамять дискретного контроллера', 'Видеопамять дискретного контроллера ГБ', 'Влагозащитный корпус', 'Внешняя поверхность', 'Возможность апгрейда', 'Возможность апгрейда HDD/SSD', 'Возможность апгрейда оперативной памяти', 'Возможность зарядки от Type-C (USB Power Delivery)', 'Возможность передачи видео через USB Type-C', 'Встроенная солнечная батарея', 'Встроенный TV-тюнер', 'Встроенный кард-ридер', 'Встроенный микрофон', 'Встроенный сабвуфер', 'Входное напряжение (макс)', 'Входное напряжение (мин)', 'Высота (максимальная)', 'Высота (минимальная)', 'Высота упаковки (ед)', 'Габариты упаковки (ед) ДхШхВ', 'Глубина товара', 'Год релиза', 'Диагональ экрана в дюймах', 'Длина упаковки (ед)', 'Для геймеров', 'Для дизайна', 'Д

## Task 2. Обработка датафрейма с 4 категориями и характеристиками

In [8]:
import pandas as pd

file_path = '/home/jupyter/datasphere/project/models_df.xlsx'

# Читаем все листы Excel-файла в один словарь
sheets_dict = pd.read_excel(file_path, sheet_name=None)


artikul_dict = {}

# Проходим по всем листам, заполняем словарь, где ключ - артикул, а значение - словарь характеристик по артикулу
for sheet_name, sheet_data in sheets_dict.items():
    
    sheet_data['Код товара'] = sheet_data['Код товара'].astype(str)
    
    for artikul in sheet_data['Код товара'].unique():
        artikul_data = sheet_data[sheet_data['Код товара'] == artikul]
        characteristics = {}
        for index, row in artikul_data.iterrows():
            characteristic_name = row['Название характеристики']
            characteristic_value = row['Значение характеристики (таргет)']
            characteristics[characteristic_name] = characteristic_value
        artikul_dict[artikul] = characteristics

In [9]:
# Функция получения всех характеристик по артикулу, обработка неизвестных артикулов
def get_characteristics_by_artikul(artikul, artikul_dict):
    artikul = str(artikul)
    return artikul_dict.get(artikul, "Артикул не найден")

## Подключение YandexGPT API

In [10]:
# Функция для получения токена от YandexGPT - подключение API
def get_yandex_gpt_token():
    service_account_id = "ajelnu35h5rleak8ttco"
    key_id = "ajeqbm3poihdm8br05l3"
    private_key = os.environ['YandexGPT-IAM-Key']
    now = int(time.time())
    payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

    encoded_token = jwt.encode(
        payload,
        private_key,
        algorithm='PS256',
        headers={'kid': key_id})

    url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
    x = requests.post(url, headers={'Content-Type': 'application/json'}, json={'jwt': encoded_token}).json()
    return x['iamToken']

## Профессиональный промпт для технического описания

In [11]:
# metaprompt = """
# Сегодня вы будете писать структурированное и подробное маркетинговое описание технологического продукта.
# Исходные данные будут включать название продукта и различные технические характеристики.
# Ваша цель - создать четкое, сжатое и всеобъемлющее описание, подходящее для маркетинговых целей.
# В описании следует выделить уникальные преимущества продукта, подробно объяснить одну ключевую особенность и кратко упомянуть другие особенности.
# Кроме того, упростите технические термины для лучшего понимания и подчеркните возможности продукта по решению проблем.
# <Task Instruction Example>
# <Task>
# Создать маркетинговое описание товара.
# </Task>
# <Inputs>
# {SPECS}
# </Inputs>
# <Instructions>
# Вы будете создавать маркетинговое описание товара на основе предоставленных характеристик.
# Когда я напишу "НАЧАТЬ ОПИСАНИЕ:", вы приступите к выполнению этой задачи. Вся дальнейшая информация от "Инструктора:" будет касаться конкретного товара.
# Вот важные правила для выполнения задачи:
# - Текст должен быть написан на русском языке.
# - Объем текста должен быть от 1500 до 3000 символов.
# - Пожалуйста, напишите ответ, используя правильную разметку Markdown, чтобы он выглядел красиво и был правильно структурирован. Убедитесь, что списки правильно оформлены с использованием символов `*` или `-`. Избегайте лишних символов, таких как \n и лишних *.
# Пример структуры, которую я ожидаю:
# ## Заголовок
# Описание
# ### Основные характеристики:
# - Характеристика 1
# - Характеристика 2
# ### Преимущества:
# - Преимущество 1
# - Преимущество 2
# - Текст должен быть структурированным и персонализированным, с вежливыми обращениями к покупателю («вы», «вам», «ваши»), но без злоупотребления обращениями и написания их с большой буквы.
# - Заголовки должны:
#   - Раскрывать уникальное торговое предложение (УТП) или выгоды клиента.
#   - Упоминать «боли» клиента и товар как их решение.
#   - Призывать к действию.
#   - Быть эмоционально заряженными.
#   - Содержать вопросы к аудитории.
# - В тексте необходимо информировать о назначении товара (какую проблему клиента решает товар?) хотя бы одним сильным тезисом.
# - Кратко обозначить особенности и/или преимущества. В абзацах под заголовками подробно раскрыть наиболее важные преимущества или дать наиболее важную информацию.
# - Раскрыть ключевые преимущества бренда, коллекции и/или подхода к производству (например, «зеленое» производство, отсутствие тестирования на животных, социально ответственное производство, благотворительные программы от продажи данного товара), если производитель действительно следит за этим.
# - Аккуратно работать с возражениями, например, если телефон имеет маленький объем оперативной памяти, но большой объем встроенной памяти, сделать акцент на количестве встроенной памяти.
# Структура текста:
# - Абзацы размером не более 3-4 предложений каждый.
# - Использование маркированных и/или нумерованных списков.
# - В начале описания указать название товара – связку «Тип товара» + «Бренд» + «Модель».
# - Способ применения, состав, комплектация должны быть вынесены в отдельные характеристики.
# Недопустимые действия:
# - Не преподносить обычное свойство товара в качестве преимущества. Если у товара нет отстройки от конкурентов по этому параметру, не нужно выдавать его за особенность. Если товар сделан из пластика, не нужно писать «сделан из высококачественного пластика». Упомяните материал корпуса только если клиенты подчеркивают его надежность или он важен для совершения покупки.
# - Не вызывать ярко выраженные отрицательные эмоции у покупателя, не стыдить и не запугивать его. Акцент должен быть сделан на благополучном решении проблемы и повышении качества жизни.
# - Не использовать более 2-3 восклицательных знаков на весь текст. Текст должен быть позитивным, но не экзальтированным.
# - Уникальность описания должна быть не менее 80%.
# Стоп-слова (бан-ворды):
# - «скидка», «распродажа», «дешевый», «подарок» (кроме подарочных категорий), «бесплатно», «акция», «специальная цена», «новинка», «new», «аналог», «заказ», «хит».
# Исключаем штампы (не использовать в генерации ответа):
# - Отличается высокой прочностью и длительной эксплуатацией.
# - Максимально комфортный дизайн.
# - Осуществляется надежная защита.
# - Окажется хорошим выбором.
# - Окажется прекрасным приобретением.
# - Наслаждайтесь непревзойденным качеством.
# - Привлекательный внешний вид.
# - Прекрасное приобретение.
# - Незаменимое дополнение.
# - Незабываемые впечатления от просмотра.
# - В самых непредвиденных условиях.
# - Длительный срок эксплуатации.
# - Среди достоинств можно назвать…
# - Лучший выбор для вас.
# Пустые оценки (не использовать в генерации ответа):
# - Красивый.
# - Мощный.
# - Удобный.
# - Высококлассный.
# - Достойный.
# - Инновационный.
# - Внушительный.
# - Высококачественный.
# - Высококлассный.
# - Гарантированный.
# - Эксклюзивный.
# Излишние усилители (не использовать в генерации ответа):
# - Абсолютно.
# - Невероятно.
# - Предельно.
# - Максимально.
# - Безусловно.
# - Совершенно.
# Избегаем вводных конструкций (не использовать в генерации ответа):
# - Как известно.
# - Не секрет.
# - Между прочим.
# - Очевидно.
# - Вдобавок.
# - Естественно.
# Обобщающие и неопределённые (не использовать в генерации ответа):
# - В некоторых случаях.
# - Во всех отношениях.
# - Разнообразный.
# - Различного рода.
# - В целом.
# - В среднем.
# Паразиты времени (не использовать в генерации ответа):
# - Сегодня.
# - Сейчас.
# - В настоящее время.
# - В наши дни.
# - В современном мире.
# Исключи уточнения в конце по типу (не использовать в генерации ответа):
# - Обратите внимание, что цены и характеристики могут меняться, поэтому перед покупкой рекомендуется уточнить актуальную информацию у продавца.
# Разделённые конструкции:
# - Плохо: Для планшета подойдёт чехол, который имеет легкое магнитное крепление.
# - Хорошо: Для планшета подойдёт чехол с лёгким магнитным креплением.
# Страдательный залог:
# - Плохо: Механизм оборудован 6 шарикоподшипниками, обеспечивающими плавный ход.
# - Хорошо: Механизм оборудован 6 подшипниками для плавного хода.
# Сложные предложения:
# - Плохо: Тонкий, но практичный чехол Tab S предлагает магнитное крепление для фиксации, клавиатура у него съемного типа, так что изделие сможет выступить просто в роли чехла.
# - Хорошо: Tab S - тонкий, но практичный чехол с магнитным креплением для фиксации. При необходимости клавиатуру можно снять и оставить только заднюю часть чехла.
# Ваша задача заключается в написании маркетингового описания товара на основе предоставленных характеристик:
# <characteristics>
# {SPECS}
# </characteristics>
# НАЧАТЬ ОПИСАНИЕ:
# <description>
# <instructions>
# """

In [12]:
metaprompt = """Сегодня вы будете писать структурированное и подробное маркетинговое описание технологического продукта. Исходные данные будут включать только характеристики товара в структурированном словаре (ключ: значение). 
Ваша цель — создать четкое, сжатое и всеобъемлющее описание, подходящее для маркетинговых целей. В описании следует выделить уникальные преимущества продукта, подробно объяснить одну ключевую особенность и кратко упомянуть другие особенности. 
Упростите технические термины для лучшего понимания и подчеркните возможности продукта по решению проблем.
Когда я напишу "НАЧАТЬ ОПИСАНИЕ", вы приступите к выполнению этой задачи.
Исходные данные, словарь, где ключ - название характеристики, а значение - сама характеристика товара:
{SPECS}
Важные правила для выполнения задачи:

Текст должен быть написан на русском языке.
Объем текста должен быть от 1500 до 3000 символов.
Напишите ответ, используя правильную разметку Markdown, чтобы он выглядел красиво и был правильно структурирован. Убедитесь, что списки правильно оформлены с использованием символов * или -. Избегайте лишних символов, таких как \n и лишних *.

Структура текста:

Название товара:
Связка «Тип товара» + «Модель».

Описание:
Персонализированное обращение к покупателю («вы», «вам», «ваши»).
Информирование о назначении товара (какую проблему клиента решает товар?) хотя бы одним сильным тезисом.

Основные характеристики:
Список ключевых характеристик.

Преимущества:
Раскрытие ключевых преимуществ товара.

Дополнительная информация:
Кратко обозначьте особенности и/или преимущества.
В абзацах под заголовками подробно раскройте наиболее важные преимущества или дайте наиболее важную информацию.
Аккуратная работа с возражениями: например, если телефон имеет маленький объем оперативной памяти, но большой объем встроенной памяти, сделайте акцент на количестве встроенной памяти.

Заключение:
Подчеркните, как товар улучшает качество жизни покупателя.

Недопустимые действия:
Не преподносить обычное свойство товара в качестве преимущества. Если у товара нет отстройки от конкурентов по этому параметру, не нужно выдавать его за особенность.
Не вызывать ярко выраженные отрицательные эмоции у покупателя, не стыдить и не запугивать его. Акцент должен быть сделан на благополучном решении проблемы и повышении качества жизни.
Не использовать более 2-3 восклицательных знаков на весь текст. Текст должен быть позитивным, но не экзальтированным.

Уникальность описания должна быть не менее 80%.

Стоп-слова (не использовать в генерации ответа):
«скидка», «распродажа», «дешевый», «подарок» (кроме подарочных категорий), «бесплатно», «акция», «специальная цена», «новинка», «new», «аналог», «заказ», «хит».

Исключаем штампы (не использовать в генерации ответа):
Отличается высокой прочностью и длительной эксплуатацией.
Максимально комфортный дизайн.
Осуществляется надежная защита.
Окажется хорошим выбором.
Окажется прекрасным приобретением.
Наслаждайтесь непревзойденным качеством.
Привлекательный внешний вид.
Прекрасное приобретение.
Незаменимое дополнение.
Незабываемые впечатления от просмотра.
В самых непредвиденных условиях.
Длительный срок эксплуатации.
Среди достоинств можно назвать…
Лучший выбор для вас.

Пустые оценки (не использовать в генерации ответа):
Красивый.
Мощный.
Удобный.
Высококлассный.
Достойный.
Инновационный.
Внушительный.
Высококачественный.
Высококлассный.
Гарантированный.
Эксклюзивный.

Излишние усилители (не использовать в генерации ответа):
Абсолютно.
Невероятно.
Предельно.
Максимально.
Безусловно.
Совершенно.

Избегаем вводных конструкций (не использовать в генерации ответа):
Как известно.
Не секрет.
Между прочим.
Очевидно.
Вдобавок.
Естественно.

Обобщающие и неопределённые (не использовать в генерации ответа):
В некоторых случаях.
Во всех отношениях.
Разнообразный.
Различного рода.
В целом.
В среднем.

Паразиты времени (не использовать в генерации ответа):
Сегодня.
Сейчас.
В настоящее время.
В наши дни.
В современном мире.

Исключите уточнения в конце по типу (не использовать в генерации ответа):
Обратите внимание, что цены и характеристики могут меняться, поэтому перед покупкой рекомендуется уточнить актуальную информацию у продавца.

Разделённые конструкции:
Плохо: Для планшета подойдёт чехол, который имеет легкое магнитное крепление.
Хорошо: Для планшета подойдёт чехол с лёгким магнитным креплением.

Страдательный залог:
Плохо: Механизм оборудован 6 шарикоподшипниками, обеспечивающими плавный ход.
Хорошо: Механизм оборудован 6 подшипниками для плавного хода.

Сложные предложения:
Плохо: Тонкий, но практичный чехол Tab S предлагает магнитное крепление для фиксации, клавиатура у него съемного типа, так что изделие сможет выступить просто в роли чехла.
Хорошо: Tab S — тонкий, но практичный чехол с магнитным креплением для фиксации. При необходимости клавиатуру можно снять и оставить только заднюю часть чехла.

Ваша задача заключается в написании маркетингового описания товара на основе предоставленных характеристик. 

Пример выполнения и создания маркетингово описания продукта, за пример взят ноутбук Huawei MateBook D 15 BoDE-WDH9:

Ноутбук Huawei MateBook D 15 BoDE-WDH9 имеет безрамочный FHD-дисплей с полезной площадью 87%, благодаря чему на нем отображается больше информации — можно комфортно просматривать веб-страницы или работать в графических редакторах. За счет диагонали 15,6 дюймов и широкоформатного соотношения сторон можно не только смотреть фильмы или видеоролики, но и редактировать большие таблицы. IPS-матрица отличается реалистичной цветопередачей и широкими углами обзора. Функции защиты зрения позволяют проводить за компьютером часы напролет, не чувствуя усталости.
Технология «Мультискрин» дает возможность проецировать дисплей смартфона прямо на ноутбук. При этом благодаря поддержке многозадачности одновременно можно разместить три окна с разными приложениями.
Особенности
стереодинамики;
сканер отпечатка пальца для входа в учетную запись без ввода пароля;
быстрая зарядка.
HD-камера для общения по видеосвязи встроена в клавиатуру и выдвигается при нажатии. Для выхода в интернет используется модуль Wi-Fi 6, что позволяет загружать и скачивать файлы со скоростью 2,4 Гбит/с. При этом, по сравнению с предыдущим поколением, сигнал более стабильный, а покрытие сети шире. По Bluetooth-соединению можно подключить беспроводную аудиогарнитуру и другие совместимые устройства.
Конфигурация
4-ядерный процессор Core i5 и 8 Гб оперативной памяти DDR4 гарантируют высокий уровень производительности при выполнении ресурсоемких задач. Даже если открыть десятки вкладок в браузере, ничего не будет зависать или тормозить. За обработку компьютерной графики отвечает интегрированная видеокарта Iris Xe. Твердотельный SSD-накопитель объемом 256 Гбайт ускоряет загрузку системы и обеспечивает мгновенный доступ к файлам.
Автономность
Литий-полимерная аккумуляторная батарея энергоемкостью 56 Wh способна продержаться без подзарядки на протяжении 12 часов, в зависимости от нагрузки. Поэтому, отправляясь на работу или учебу, зарядный блок питания можно оставить дома.

НАЧАТЬ ОПИСАНИЕ
"""

In [13]:
def format_prompt(metap, specs_list):
    return metap.format(SPECS=specs_list)

## Создание запроса в YandexGPT, получение ответа в виде markdown

In [14]:
# Функция для обработки запроса YandexGPT
def get_gpt_response(formatted_prompt, token):
    url = 'https://llm.api.cloud.yandex.net/foundationModels/v1/completion'

    data = {
        'modelUri': 'gpt://b1g60k1kva2jrejf255o/yandexgpt/latest',
        'completionOptions': {'stream': False, 'temperature': 0.2, 'maxTokens': 2500},
        'messages': [
            {"role": "system", "text": formatted_prompt},
            {"role": "user", "text": formatted_prompt}
        ]
    }

    response = requests.post(url, headers={'Authorization': 'Bearer ' + token}, json=data).json()
    return response

In [15]:
# Функция для обработки формата markdown, исправление явных ошибок генерации
def format_markdown(text):
    
    text = text.replace('\\n', '\n')
    text = re.sub(r'\* ', '* ', text)
    
    formatted_text = text.strip()
    
    return formatted_text

## Промпт с учетом техник SEO-оптимизации для саммари

In [26]:
metaprompt_summary = """Сегодня вы будете писать SEO-оптимизированное саммари для технологического продукта. 
Ваша цель — создать краткое, информативное и привлекательное описание, подходящее для маркетинговых целей и оптимизированное для поисковых систем. 
Используйте ключевые слова, структурированные данные и улучшите пользовательский опыт. Будьте очень креативным, чтобы описание выглядело живым и ценным для пользователей.
Самое главное - умести красочное, креативное, живое описание в диапазон от 250 до 750 символов, чтобы оно было полноценным SEO-описанием для продукта в поисковике.
Не используй значения характеристик, выделяй лишь уникальность продукта, его плюсы, но не используй числовые значения характеристик в ответе.
Когда я напишу "НАЧАТЬ ОПИСАНИЕ", вы приступите к выполнению этой задачи. Никогда не пиши в конце ответа это: "Обратите внимание, что цены и характеристики могут меняться, поэтому перед покупкой рекомендуется уточнить актуальную информацию у продавца."
Исходные данные - словарь со всеми характеристиками товара, где ключ - название характеристики, а значение - сама характеристика:
{SPECS}
Важные правила для выполнения задачи:
Текст должен быть написан на русском языке.
Объем текста должен быть от 250 до 750 символов.
Напишите ответ, используя правильную разметку Markdown, чтобы он выглядел красиво и был правильно структурирован. Убедитесь, что списки правильно оформлены с использованием символов * или -. Избегайте лишних символов, таких как \n и лишних *.
Структура текста:
Название товара:

Включите название и основное ключевое слово.
Описание:

Используйте ключевые слова.
Информируйте о назначении товара и его ключевых особенностях.
Основные характеристики:

Список ключевых характеристик с использованием ключевых слов. Не давайте слишком много информации про характеристики, это ведь не техническое описание. Упомяните лишь уникальные, которые оценят пользователи.

Заключение:
Подчеркните, как товар улучшает качество жизни покупателя.

Недопустимые действия:
Не преподносить обычное свойство товара в качестве преимущества. Если у товара нет отстройки от конкурентов по этому параметру, не нужно выдавать его за особенность.
Не вызывать ярко выраженные отрицательные эмоции у покупателя, не стыдить и не запугивать его. Акцент должен быть сделан на благополучном решении проблемы и повышении качества жизни.
Не использовать более 2-3 восклицательных знаков на весь текст. Текст должен быть позитивным, но не экзальтированным.
Уникальность описания должна быть не менее 80%.

Стоп-слова (не использовать в генерации ответа):
«скидка», «распродажа», «дешевый», «подарок» (кроме подарочных категорий), «бесплатно», «акция», «специальная цена», «новинка», «new», «аналог», «заказ», «хит».

Исключаем штампы (не использовать в генерации ответа):
Отличается высокой прочностью и длительной эксплуатацией.
Максимально комфортный дизайн.
Осуществляется надежная защита.
Окажется хорошим выбором.
Окажется прекрасным приобретением.
Наслаждайтесь непревзойденным качеством.
Привлекательный внешний вид.
Прекрасное приобретение.
Незаменимое дополнение.
Незабываемые впечатления от просмотра.
В самых непредвиденных условиях.
Длительный срок эксплуатации.
Среди достоинств можно назвать…
Лучший выбор для вас.

Пустые оценки (не использовать в генерации ответа):
Красивый.
Мощный.
Удобный.
Высококлассный.
Достойный.
Инновационный.
Внушительный.
Высококачественный.
Высококлассный.
Гарантированный.
Эксклюзивный.

Излишние усилители (не использовать в генерации ответа):
Абсолютно.
Невероятно.
Предельно.
Максимально.
Безусловно.
Совершенно.

Избегаем вводных конструкций (не использовать в генерации ответа):
Как известно.
Не секрет.
Между прочим.
Очевидно.
Вдобавок.
Естественно.
Обобщающие и неопределённые (не использовать в генерации ответа):
В некоторых случаях.
Во всех отношениях.
Разнообразный.
Различного рода.
В целом.
В среднем.

Паразиты времени (не использовать в генерации ответа):
Сегодня.
Сейчас.
В настоящее время.
В наши дни.
В современном мире.

Исключите уточнения в конце по типу (не использовать в генерации ответа):
Обратите внимание, что цены и характеристики могут меняться, поэтому перед покупкой рекомендуется уточнить актуальную информацию у продавца.

Разделённые конструкции:
Плохо: Для планшета подойдёт чехол, который имеет легкое магнитное крепление.
Хорошо: Для планшета подойдёт чехол с лёгким магнитным креплением.

Страдательный залог:
Плохо: Механизм оборудован 6 шарикоподшипниками, обеспечивающими плавный ход.
Хорошо: Механизм оборудован 6 подшипниками для плавного хода.

Сложные предложения:
Плохо: Тонкий, но практичный чехол Tab S предлагает магнитное крепление для фиксации, клавиатура у него съемного типа, так что изделие сможет выступить просто в роли чехла.
Хорошо: Tab S — тонкий, но практичный чехол с магнитным креплением для фиксации. При необходимости клавиатуру можно снять и оставить только заднюю часть чехла.

Пример:

Название товара:
Samsung Galaxy S24 Ultra

Описание:
Добро пожаловать в эру мобильного AI. С Galaxy S24 Ultra в руках вы сможете раскрыть совершенно новые уровни творчества, продуктивности и возможностей, начиная с самого важного устройства в вашей жизни. С вашего смартфона.

Основные характеристики:
Новые цвета. Минеральный блеск
Встроенный стилус S Pen пишет совершенно новую главу
Высококлассный NPU. Лучшая в отрасли камера. Теперь с AI.
Длительное время работы от аккумулятора.
200 МП.
Съемка деталей, не уступающая реальности

Заключение:
Переносите данные просто с iOS на Android. Откройте новые горизонты AI. Забудьте о барьерах. Делитесь файлами мгновенно. Попробуйте Galaxy на своем телефоне. С Galaxy вы начнёте видеть жизнь в новом цвете.

НАЧАТЬ ОПИСАНИЕ
"""

## Настройка графического интерфейса Gradio, использование всех функций

In [22]:
import gradio as gr

def generate_description(artikul):
    artikul = artikul.replace(' ', '')
    specs_list = get_characteristics_by_artikul(artikul, artikul_dict)
    
    if specs_list == "Артикул не найден":
        return "Артикул не найден", ""
    
    # Форматируем и отправляем запрос для маркетингового описания
    formatted_prompt = format_prompt(metaprompt, specs_list)
    token = get_yandex_gpt_token()
    try:
        response = get_gpt_response(formatted_prompt, token)
        marketing_description = response["result"]["alternatives"][0]["message"]["text"]
        marketing_description = format_markdown(marketing_description)
    except requests.exceptions.Timeout:
        return "Ошибка: запрос превысил время ожидания", ""
    except Exception as e:
        return f"Ошибка: {str(e)}", ""
    
    # Форматируем и отправляем запрос для SEO-оптимизированного саммари
    formatted_summary_prompt = format_prompt(metaprompt_summary, specs_list)
    try:
        response = get_gpt_response(formatted_summary_prompt, token)
        seo_summary = response["result"]["alternatives"][0]["message"]["text"]
        seo_summary = format_markdown(seo_summary)
    except requests.exceptions.Timeout:
        return marketing_description, "Ошибка: запрос превысил время ожидания"
    except Exception as e:
        return marketing_description, f"Ошибка: {str(e)}"
    
    return marketing_description, seo_summary

# Gradio интерфейс
with gr.Blocks() as iface:
    gr.Markdown("# Маркетинговое описание и саммари товара")
    gr.Markdown("### Введите артикул (код товара Ситилинк), чтобы получить маркетинговое описание и SEO-оптимизированное саммари.")
    
    with gr.Row():
        artikul_input = gr.Textbox(label="Артикул", placeholder="Введите артикул здесь...", lines=1, max_lines=1)
        submit_button = gr.Button("Сгенерировать описание")
    
    with gr.Tab("Результаты"):
        with gr.TabItem("Маркетинговое описание"):
            markdown_output = gr.Markdown()
        with gr.TabItem("SEO-оптимизированное саммари"):
            summary_output = gr.Markdown()
    
    gr.Markdown("---")
    gr.Markdown("Пример использования:")
    gr.Markdown("1. Введите любой код товара из Excel-файла в текстовое поле. (например: 2004835 - Macbook Pro, 1907743 - Haier Smart TV, 1923689 - Samsung S22, 1903468 - LG UltraWide 34)\n2. Нажмите кнопку 'Сгенерировать описание'.\n3. Получите тех. описание и саммари в формате Markdown.")

    submit_button.click(fn=generate_description, inputs=artikul_input, outputs=[markdown_output, summary_output])

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
IMPORTANT: You are using gradio version 4.26.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://7d6d95bbdee4bc98c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Дообучение модели YandexGPT

In [18]:
specs = get_characteristics_by_artikul("2010075", artikul_dict)
fprompt = format_prompt(metaprompt, specs)
fsumprompt = format_prompt(metaprompt_summary, specs)

In [19]:
print(fsumprompt)

Сегодня вы будете писать SEO-оптимизированное саммари для технологического продукта. 
Ваша цель — создать краткое, информативное и привлекательное описание, подходящее для маркетинговых целей и оптимизированное для поисковых систем. 
Используйте ключевые слова, структурированные данные и улучшите пользовательский опыт. Будьте очень креативным, чтобы описание выглядело живым и ценным для пользователей.

Когда я напишу "НАЧАТЬ ОПИСАНИЕ", вы приступите к выполнению этой задачи.
Исходные данные - словарь со всеми характеристиками товара, где ключ - название характеристики, а значение - сама характеристика:
{'PartNumber/Артикул Производителя': '75C845', 'Акустическая схема': '2.1', 'Антенный вход': '2', 'Безрамочный дизайн': 'нет', 'Бренд': 'TCL', 'Веб-камера': 'нет', 'Версия операционной системы': 'Google TV', 'Вес без подставки': '32.6', 'Вес товара': '35.8', 'Вес упаковки (ед)': '13.5', 'Возможность записи на USB носитель': 'ДА', 'Время отклика пикселя': '6.5', 'Встроенный Bluetooth': 'Д